In [4]:
import os
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

In [5]:
training_data_uri = 's3://covid-19-claims/claims.csv'

In [6]:
!pygmentize './use_classifier.py'

import tensorflow as tf
import tensorflow_hub as hub
import argparse
import os
import numpy as np
import json
from sklearn.model_selection import train_test_split

def model(x_train, y_train, x_test, y_test):
    hub_model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
    hub_layer = hub.KerasLayer(hub_model, output_shape=[20], input_shape=[], 
                           dtype=tf.string, trainable=True)
    
    model = tf.keras.Sequential()
    model.add(hub_layer)
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(1))

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(x_train, y_train)
    model.evaluate(x_test, y_test)

    return model


def _parse_args():
    parser = argparse.ArgumentParser()

    # Data, model, and output directories
    # model_dir is always passed in from SageMaker. By default this is a S3 path under the

In [13]:
use_estimator = TensorFlow(entry_point='use_classifier.py',
                             role=role,
                             train_instance_count=1,
                             train_instance_type='ml.m5.xlarge',
                             framework_version='2.1.0',
                             py_version='py3',
                             distributions={'parameter_server': {'enabled': True}})

Parameter distribution will be renamed to {'parameter_server': {'enabled': True}} in SageMaker Python SDK v2.


In [14]:
use_estimator.fit(training_data_uri)

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-06-19 00:26:49 Starting - Starting the training job...
2020-06-19 00:26:52 Starting - Launching requested ML instances......
2020-06-19 00:27:58 Starting - Preparing the instances for training......
2020-06-19 00:29:09 Downloading - Downloading input data
2020-06-19 00:29:09 Training - Downloading the training image..2020-06-19 00:29:27,609 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-06-19 00:29:27,616 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-06-19 00:29:27,942 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-06-19 00:29:27,957 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-06-19 00:29:27,971 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-06-19 00:29:27,980 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {
        "sagemaker_parame

UnexpectedStatusException: Error for Training job tensorflow-training-2020-06-19-00-26-48-821: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/usr/bin/python3 use_classifier.py --model_dir s3://sagemaker-us-west-2-378706508412/tensorflow-training-2020-06-19-00-26-48-821/model"

In [ ]:
# ml.m5.xlarge
# ml.p2.xlarge - accelerated compute
predictor = use_estimator.deploy(initial_instance_count=1, instance_type='ml.p2.xlarge')